# **Traffic Incidents optimization**

## **Context**

In some high-density traffic cities like Bogotá, slow attention to traffic incidents remains an issue, and a method to deal with it is yet to be implemented. Quick response is important for most drivers. Those involved in accidents would have higher chances of survival, and even those never involved in an incident would see the benefit of faster moving traffic.

Let us define what we mean by traffic incident or accident. Firstly, an accident implies the happening of a crash, which could be one of a car against an object, or between vehicles. On the other hand, incident is a more general term, it refers to any event that negatively affects traffic flow, so in those terms, an accident is a specific kind of incident.

In 2019, Bogotá had the alarming rate of a traffic accident every five to six minutes, and in 2018 there were a total of 50 800 traffic incidents, a number excessively high, that if it were to be reduced, a great quantity of time would be saved. This of course puts interest on projects trying to fix this problem, which can take many different approaches and focal points. Among these are 



## **Objectives**

## **Model formulation**

El modelo para la solución a nuestro problema se divide en 2 partes, la primera parte  se encarga de  maximizar los pesos totales  de la gravedad de los incidentes que se abordarán con los recursos de respuesta actualmente  disponible, y la  segunda parte se encarga de minimizar el tiempo total de viaje de la respuesta de todas las unidades.

$\textbf{Fase 1:}$
La función objetivo en esta fase es: 

$(1a)$ $$\max \sum_{j=1}^{N_i}a_{s(j)}z_j$$

bajo las restricciones:

$(1b)$ $$\sum_{i=1}^{N_t} x_{ij}\geq b_{s(j)} z_j,\hspace{1cm} \forall j\in N_i$$ 

$(1c)$  $$\sum_{i=1}^{N_i} x_{ij}\leq 1,\hspace{1cm} \forall i\in RU $$ 

$(1d)$  $$x_{ij}=0 \text{ o } 1, \hspace{1cm} \forall i\in RU, \forall j\in N_i$$

$(1e)$   $$z_{j}=0 \text{ o } 1, \hspace{1cm}, \forall j\in N_i$$

$\textbf{Fase 2: }$ La función objetivo en esta parte es:

$(2a)$ $$\min \sum_{i=1}^{N_t}\sum_{j=1}^{N_i} x_{ij}t_{ij}(t)$$

bajo las restricciones: 

$(2b)$ $$\sum_{i=1}^{N_t} x_{ij}\geq b_{s(j)} z_j,\hspace{1cm} \forall j\in N_i$$ 

$(2c)$  $$\sum_{i=1}^{N_i} x_{ij}\leq 1,\hspace{1cm} \forall i\in RU $$ 

$(2d)$  $$x_{ij}=0 \text{ o } 1, \hspace{1cm} \forall i\in RU, \forall j\in N_i$$


Donde 

$t_{ij}(t): $ tiempo de viaje del camino más corte entre los puntos $i$, $j$ en el paso de tiempo $t$

$S(j): $ La función que devuelve la prioridad de un incidente dado $j$ (este es un número entre 1 y 4, donde 1 es la menor prioridad)

$a_l: $ El peso de la gravedad de los incidentes con l-ésima prioridad

$b_l:$ El número de grúas necesarias para ser despachadas a un incidente con l-ésima prioridad(se necesitan 2 para una prioridad de 4, 1 para una prioridad de 3, 1 para una prioridad de 2, y 0 para una prioridad de 1)

$N_i:$ El número de incidentes que están esperando grúas 

$N_t:$ El número de grúas en todo en sistema

$RU: $ Unidades de respuesta 


Variables de decisión

$z_j=1$  Si el incidente j-ésimo será tratado por ciertas grúas, de lo contrario 0

$x_{ij}=1$ Si la grúa i se despacha al incidente j, de lo contrario 0 

En la fase 1 del modelo, la función objetivo (1a) es maximizar los pesos totales de la gravedad de los incidentes a tratar
sin exceder la limitación de número de unidades de respuesta  disponibles. La restricción (1b) garantiza que se envíe una cantidad suficiente
de grúas a cada lugar del incidente. La restricción (1c) indica que una grúa solo puede atender un solo incidente en un
momento dado. Las restricciones (1d) y (1e) representan restricciones de enteros, la ecuación (2a) se encarga de minimizar el tipo de viaje estimado de las unidades que se requieren para moverse desde sus ubicaciones actuales a las ubicaciones de los incidentes asignados.

Este problema pertenece a la programación lineal entera (puesto que la función objetivo es lineal, tanto en la fase 1 como en la fase 2 es un producto punto entre 2 vectores, y las restricciones son lineales y las variables de decisión son entera), dado que este problema en el fondo es lineal tiene solución por medio de algoritmos que involucren convexidad, pero por las restricciones enteras resulta que este problema no siempre va a tener solución de hecho es un problema NP-completo.

## **Model implementation**

Imports

In [1]:
import cvxpy as cp
import numpy as np
import pandas as pd
# np.set_printoptions(precision=3, suppress=True)

First step is to maximize the severities of the incidents under current response resources(Number of available agents)

In [19]:
t = pd.read_csv("durations.csv").iloc[:,1:]
i = t.shape[0] # number of agents
j = t.shape[1] # number of incidents

b = np.array([np.random.randint(1, 4) for i in range(j)])

In [24]:
x = cp.Variable((i, j), boolean=True) # agents-incidents matrix
z = cp.Variable(j, boolean=True) # incidents coverage vector

objective = cp.Maximize(b@z) 

constraints = [cp.sum(x, axis=0) >= cp.multiply(b, z), # required agents per incidents
               cp.sum(x, axis=1) <= 1 # one agent covers at max. one incident
               ]
               
problem = cp.Problem(objective, constraints)

problem.solve(verbose=0)

100.0

In [26]:
print("Agents :\n", x.value, end="\n\n")
print("Incidents:\n", z.value)

Agents :
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]

Incidents:
 [0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0.
 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0.
 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 1. 0. 0.

Next step is to minize the total travel time of all the agents

In [29]:
objective = cp.Minimize(cp.sum(cp.sum(cp.multiply(x, t), axis=0)))


constraints = [cp.sum(x, axis=0) >= cp.multiply(b, z.value), # required agents per incidents
               cp.sum(x, axis=1) <= 1 # one agent covers at max. one incident
               ]
               
problem = cp.Problem(objective, constraints)

problem.solve()

56575.899999999994

In [34]:
np.sum(x.value, axis=1)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

## **Model in practice**

In [ ]:
import pandas as pd

In [35]:
agents = pd.read_csv("agents_may.csv", parse_dates=["read_time", "time_stamp"])
incidents = pd.read_csv("incidents_may.csv", parse_dates=["incident_time"])

In [ ]:
agents.head()

,id,dev_id,read_time,time_stamp,speed,grp_name,localidad,latitude,longitude,devicegroupid
0,1651381596,868033050089715,2022-05-01 00:06:36,2022-05-01 00:01:40,0.053087,S60,RAFAEL URIBE URIBE,4.551646,-74.108515,6
1,1651381596,868033050102534,2022-05-01 00:06:36,2022-05-01 00:01:42,0.000000,S60,KENNEDY,4.650030,-74.147322,6
2,1651381596,868033050090085,2022-05-01 00:06:36,2022-05-01 00:01:47,0.000000,S60,RAFAEL URIBE URIBE,4.581688,-74.128348,6
3,1651381596,868033050103508,2022-05-01 00:06:36,2022-05-01 00:01:52,0.000000,S60,CIUDAD BOLIVAR,4.551098,-74.147635,6
4,1651381596,868033050101650,2022-05-01 00:06:36,2022-05-01 00:01:55,0.000000,S60,KENNEDY,4.619840,-74.159668,6
